- Author: Ben Du
- Date: 2021-05-29 10:12:42
- Title: Error Handling in Rust
- Slug: error-handling-in-rust
- Category: Computer Science
- Tags: Computer Science, programming, error, handling, exception, try, catch, panic
- Modified: 2021-06-19 23:59:11


**Things on this page are fragmentary and immature notes/thoughts of the author. Please read with your own judgement!**

## Tips and Traps

1. The question mark `?` ...

2. `eprintln!`

3. `Result.ok` converts a `Result` object to an `Option` object.
    Symetrically,
    `Option.ok_or` converts an `Option` to a `Result` object.

3. The recommended way of error handling in Rust is 
    to  define an enum that covers meaningful error cases for your library,
    implement `Debug`, `Display` and `std::error::Error` for that enum.
    This is fast/cheap (no heap allocation on error), 
    and precise and easy to use.

4. By default errors in Rust are checked (at compile time). 
    However, 
    you can get unchecked error using `Box<dyn std::error::Error + 'static>` with `Send` and `Sync`.

5. The disscussion in
    [The state of error handling in the 2018 edition](https://users.rust-lang.org/t/the-state-of-error-handling-in-the-2018-edition/23263)
    suggests that thiserror (for libraries) + anyhow (for applications) is a good combination.
    

In [2]:
:dep thiserror = "1.0.25"
:dep anyhow = "1.0.41"

In [3]:
use thiserror::Error;

#[derive(Error, Debug)]
pub enum ParseRankError {
    #[error("{0} is not a valid symbol for card rank!")]
    InvalidSymbol(char),
    #[error("{0} is not a valid integer for card rank!")]
    InvalidInteger(u8),
}

In [4]:
let err = ParseRankError::InvalidSymbol('m');
err

InvalidSymbol('m')

In [5]:
err.to_string()

"m is not a valid symbol for card rank!"

In [6]:
println!("{}", err);

m is not a valid symbol for card rank!


In [7]:
println!("{:?}", err);

InvalidSymbol('m')


In [8]:
println!("{:?}: {}", err, err);

InvalidSymbol('m'): m is not a valid symbol for card rank!


In [9]:
eprintln!("{:?}: {}", err, err);

InvalidSymbol('m'): m is not a valid symbol for card rank!


In [12]:
use std::io;

In [16]:
let err = io::Error::new(io::ErrorKind::Other, "oh no!");
err

Custom { kind: Other, error: "oh no!" }

In [17]:
eprintln!("{:?}: {}", err, err);

Custom { kind: Other, error: "oh no!" }: oh no!


In [19]:
println!("{:?}", io::Error::last_os_error());

Os { code: 2, kind: NotFound, message: "No such file or directory" }


In [ ]:
use thiserror::Error;

/// WordCountError enumerates all possible errors returned by this library.
#[derive(Error, Debug)]
pub enum WordCountError {
    /// Represents an empty source. For example, an empty text file being given
    /// as input to `count_words()`.
    #[error("Source contains no data")]
    EmptySource,

    /// Represents a failure to read from input.
    #[error("Read error")]
    ReadError { source: std::io::Error },

    /// Represents all other cases of `std::io::Error`.
    #[error(transparent)]
    IOError(#[from] std::io::Error),
}

In [ ]:
/// WordCountError enumerates all possible errors returned by this library.
#[derive(Debug)]
enum WordCountError {
    /// Represents an empty source. For example, an empty text file being given
    /// as input to `count_words()`.
    EmptySource,

    /// Represents a failure to read from input.
    ReadError { source: std::io::Error },

    /// Represents all other cases of `std::io::Error`.
    IOError(std::io::Error),
}

impl std::error::Error for WordCountError {
    fn source(&self) -> Option<&(dyn std::error::Error + 'static)> {
        match *self {
            WordCountError::EmptySource => None,
            WordCountError::ReadError { ref source } => Some(source),
            WordCountError::IOError(_) => None,
        }
    }
}

impl std::fmt::Display for WordCountError {
    fn fmt(&self, f: &mut std::fmt::Formatter) -> std::fmt::Result {
        match *self {
            WordCountError::EmptySource => {
                write!(f, "Source contains no data")
            }
            WordCountError::ReadError { .. } => {
                write!(f, "Read error")
            }
            WordCountError::IOError(ref err) => {
                err.fmt(f)
            }
        }
    }
}

impl From<std::io::Error> for WordCountError {
    fn from(err: std::io::Error) -> WordCountError {
        WordCountError::IOError(err)
    }
}

## References 

- [thiserror](https://crates.io/crates/thiserror)
    
- [anyhow](https://crates.io/crates/anyhow)

- [Rust: Structuring and handling errors in 2020](https://nick.groenen.me/posts/rust-error-handling/)

- [RustConf 2020 - Error handling Isn't All About Errors by Jane Lusby](https://www.youtube.com/watch?v=rAF8mLI0naQ)

- [The state of error handling in the 2018 edition](https://users.rust-lang.org/t/the-state-of-error-handling-in-the-2018-edition/23263)

- [ERROR HANDLING SURVEY](https://blog.yoshuawuyts.com/error-handling-survey/)

- [Scott Wlaschin - Railway Oriented Programming — error handling in functional languages](https://vimeo.com/97344498)

- [What Makes Rust Unique Error handling](https://www.youtube.com/watch?v=c6Shucw8p48)

- [Do you agree with my opinions on error handling?](https://users.rust-lang.org/t/do-you-agree-with-my-opinions-on-error-handling/23600)

- [Error handling in library: Enum or Trait type](https://users.rust-lang.org/t/error-handling-in-library-enum-or-trait-type/53750/8)

- [What is this question mark operator about?](https://stackoverflow.com/questions/42917566/what-is-this-question-mark-operator-about)

- [Rust - Which error-handling crate to use? ErrorTypes, thiserror & anyhow](https://www.youtube.com/watch?v=UgIQo__luHw)
    
- [Should I include the error type when defining an error in Rust?](https://users.rust-lang.org/t/should-i-include-the-error-type-when-defining-an-error-in-rust/61166)